In [3]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:
df=pd.read_csv(r'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

In [7]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [16]:
df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [10]:
df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [42]:
spacex_df.groupby('class',as_index=False)['Launch Site'].value_counts()

,class,Launch Site,count
0,0,CCAFS LC-40,19
1,0,VAFB SLC-4E,6
2,0,CCAFS SLC-40,4
3,0,KSC LC-39A,3
4,1,KSC LC-39A,10
5,1,CCAFS LC-40,7
6,1,VAFB SLC-4E,4
7,1,CCAFS SLC-40,3


In [50]:
spacex_df = df
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload

9600.0

In [64]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = df
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                 html.Div([dcc.Dropdown(id='site-dropdown',
                                 options=[{'label': 'All Sites', 'value': 'ALL'},
                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                        
                value='ALL',
                placeholder="place holder here",
                searchable=True
                )]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.P("Payload range (Kg):"),dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,marks={0:'0',2500:'2500',5000:'5000',7500:'7500',10000:'10000'},
                                                                              
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site ')
        return fig
    else:
       filtered_df = spacex_df.groupby('class',as_index=False)['Launch Site'].value_counts() 
       data=filtered_df[filtered_df['Launch Site']==entered_site]
       fig = px.pie(data, values='count', 
       names='class', 
       title='Total Success Launches By Site ')
       return fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,slider_range):
    min_load,max_load=slider_range
    data= spacex_df[(df['Payload Mass (kg)']>min_load)&(df['Payload Mass (kg)']<max_load)]
    if entered_site == 'ALL':
       fig1=px.scatter(data,y='class',x='Payload Mass (kg)',color='Booster Version Category',
                       title='Correlation between Payload and success for all Sites') 
       return fig1 
    else:
       data1=data[spacex_df['Launch Site']==entered_site]
       fig1=px.scatter(data1,y='class',x='Payload Mass (kg)',color='Booster Version Category',
                       title='Correlation between Payload and success for all Sites') 
       return fig1
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8053)

1- KSC LC-39A has the largest successful launches about 41.7%
2- CCAFS SLC-40 site has the highest launch success rate about 42.9 % 
3- ranges between 2000 and 5000 has the highest launch success rate
4- ranges between 0kg and 3000 kg has the lowest rate success
5- F9 Booster version FT has the highest success rate